# Titer analysis of LASV GP pseudotyped lentivirus

This notebook analyzes data from 3 experiments:

1. Titering WT virus from transfection and recovery as well as library viruses from transfection in 293Ts with LAMP1, alpha-dystroglycan, or both added.

2. Titering library viruses from transfection.

3. Titering recovered library viruses.

In [ ]:
import os
import warnings

import pandas as pd
import numpy as np

from plotnine import *

from IPython.display import Image

In [ ]:
warnings.simplefilter('ignore')

In [ ]:
royal2 = ("#9A8822", "#F5CDB4", "#F8AFA8", "#FDDDA0", "#74A089")
CBP = ('#999999', '#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7')

In [ ]:
theme_set(theme_seaborn(style='white', context='talk', font_scale=1))

## Read in titer data

In [ ]:
dif_cells = pd.read_csv('./titer_csvs/210330_GP_DifCellLines_Titers.csv')
tfct_libs = pd.read_csv('./titer_csvs/210301_LibTiteringTableTfct_calc.csv')
rec_libs = pd.read_csv('./titer_csvs/210322_rectiters_csv.csv')

In [ ]:
# dif_cells['Virus'] = dif_cells['Virus'].astype('int')
dif_cells['Virus'] = dif_cells['Virus'].replace({1.0: 'A NoHPs',
                                                 2.0: 'Rec re-opt FJ',
                                                 3.0: 'Tfct re-opt FJ',
                                                 4.0: 'Rec re-opt FJ',
                                                 5.0: 'Tfct re-opt FJ',
                                                 6.0: 'LibA',
                                                 7.0: 'LibB',
                                                 8.0: 'LibA',
                                                 9.0: 'LibB',
                                                 10.0: '1890',
                                                 11.0: 'NoVEP',
                                                 12.0: 'B NoHPs'})
display(dif_cells.head())
display(tfct_libs.head())
display(rec_libs.head())

In [ ]:
exclude = ['A NoHPs', 'B NoHPs', 'NoVEP']

dif_cells_plot = (ggplot(dif_cells[~dif_cells['Virus'].isin(exclude)], aes(x='Virus', y='Titer', fill='Transfection')) +
                  geom_point(size=3, position=position_dodge(width=0.5, preserve='total')) +
                  scale_fill_manual(values=CBP) +
                  theme(axis_text_x=element_text(angle=90, vjust=1, hjust=0.5),
                       figure_size=(8, 4)) +
                  scale_y_log10(limits=[1e0, 1e7], breaks=[1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7]) +
                  facet_wrap('VSVG?', ncol=2,labeller={'N': 'No VSV G', 'Y': '+ VSV G'}) +
#                   geom_hline(yintercept=120, linetype='dashed', color='grey') + # LoD of 120 = 6 positive cells in 1:3 dilution
#                   geom_hline(yintercept=1e3, linetype='dashed', color='lightgreen') + 
                  ylab('TU per mL') +
                  xlab('Backbone') +
                  ggtitle('Library Titers from Transfection'))

_ = dif_cells_plot.draw()

There is no difference in titer in the different cell lines.

Note that the negative control viruses are excluded from this plot. The 1890+HPs "NoVEP" viruses had a lot of the low-level shoulder bleed-through, so titers look high (almost 1e4), but there was no actual infection. 

In [ ]:
exclude_tubes = ['A3', 'B3', 'A6', 'B6']

tfct_titers_plot = (ggplot(tfct_libs[(tfct_libs['Backbone'] != 'ZsGreen') &
                                     (~tfct_libs['TubeName'].isin(exclude_tubes))],
                           aes(x='VSVG', y='Titer', fill='Backbone')) +
                  geom_point(size=4, position=position_dodge(width=0.5, preserve='total')) +
                  scale_fill_manual(values=[royal2[4], royal2[3]]) +
                  theme(axis_text_x=element_text(angle=0, vjust=1, hjust=0.5),
                       figure_size=(4, 4)) +
                  scale_y_log10(limits=[1e0, 1e7], breaks=[1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7]) +
#                   facet_wrap('~CellsInfected', ncol=3) +
#                   geom_hline(yintercept=120, linetype='dashed', color='grey') + # LoD of 120 = 6 positive cells in 1:3 dilution
#                   geom_hline(yintercept=1e3, linetype='dashed', color='lightgreen') + 
                  ylab('transduciton units per mL') +
                  xlab('VSV G Added?') +
                  ggtitle('Library Titers from Transfection'))

_ = tfct_titers_plot.draw()
tfct_titers_plot.save('./pdf_figs/210331_lib-titers-tfct.pdf')

In [ ]:
# exclude_tubes = ['A3', 'B3', 'A6', 'B6']
sample_list = ['HPs, no dox', 'HPs + dox',
               'HPs + VSV G', 'HPs + VSV G + dox',
               'HPs + ZsGreen bb', 'HPs + ZsGreen + dox',               
               'HPs + FJ + dox', 'HPs + FJ + VSV G',
               'HPs + ZsGreen + VSV G', 'No HPs + dox']
sample_cat = pd.Categorical(rec_libs['SampleName'], categories=sample_list)

# assign to a new column in the DataFrame
rec_libs = rec_libs.assign(sample_cat = sample_cat)

rec_titers_plot_all = (ggplot(rec_libs,
                          aes(x='sample_cat', y='Titer', fill='Library', shape='Dox')) +
                  geom_point(size=5, position=position_dodge(width=0.5, preserve='total')) +
                  scale_fill_manual(values=[royal2[4], royal2[3]]) +
                  theme(axis_text_x=element_text(angle=90, vjust=1, hjust=0.5),
                       plot_title=element_text(size=30),figure_size=(10, 6)) +
                  scale_y_log10(limits=[1e0, 5e7], breaks=[1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7]) +
#                   facet_wrap('Library', ncol=2) +
                  geom_hline(yintercept=50, linetype='dashed', color='grey') + # LoD of 120 = 6 positive cells in 1:3 dilution
#                   geom_hline(yintercept=1e3, linetype='dashed', color='lightgreen') + 
                  ylab('transduciton units per mL') +
                  xlab('Sample Category') +
                  ggtitle('Library Titers from Transduced Cells'))

_ = rec_titers_plot_all.draw()
rec_titers_plot_all.save('./pdf_figs/210331_lib1-titers-rec.pdf')

In [ ]:
exclude_samples = ['HPs + ZsGreen bb', 'HPs + ZsGreen + dox','HPs + FJ + dox', 'HPs + FJ + VSV G', 'HPs + ZsGreen + VSV G', 'No HPs + dox']
rec_titers_plot_poster = (ggplot(rec_libs[~rec_libs['sample_cat'].isin(exclude_samples)],
                          aes(x='sample_cat', y='Titer', fill='Library', shape='Dox')) +
                  geom_point(size=5, position=position_dodge(width=0.5, preserve='total')) +
                  scale_fill_manual(values=[royal2[4], royal2[3]]) +
                  theme(axis_text_x=element_text(angle=90, vjust=1, hjust=0.5, size=18),
                        axis_text_y=element_text(size=18),
                        axis_title_x=element_text(size=22),
                        axis_title_y=element_text(size=22),
                        plot_title=element_text(size=24),
                        figure_size=(5, 4)) +
                  scale_y_log10(limits=[1e0, 1e7], breaks=[1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7]) +
#                   facet_wrap('Library', ncol=2) +
#                   geom_hline(yintercept=50, linetype='dashed', color='grey') + # LoD of 120 = 6 positive cells in 1:3 dilution
#                   geom_hline(yintercept=1e3, linetype='dashed', color='lightgreen') + 
                  ylab('transduciton units\nper mL') +
                  xlab('Transfection Conditions') +
                  ggtitle('Library Titers from Transduced Cells'))

_ = rec_titers_plot_poster.draw()
rec_titers_plot_poster.save('./pdf_figs/210331_lib1-titers-rec-poster.pdf')

#### 

In [ ]:
display(rec_libs[(rec_libs['sample_cat'] == 'HPs + dox') |
               (rec_libs['sample_cat'] == 'HPs + VSV G')].sort_values('Library')[['Library', 'SampleName', 'Titer']])